In [1]:
from app import Manga, MangaChapter, User, db

db.create_all()  

# Ajouter les manga à la table de base
one_piece = Manga(name = "One Piece",url = "https://www.scan-vf.net/one_piece/", cover="https://www.glenat.com/sites/default/files/images/livres/couv/9782344049020-001-T.jpeg",)
boruto = Manga(name = "Boruto", url= "https://www.scan-vf.net/boruto", cover="https://images-na.ssl-images-amazon.com/images/I/81cDaskL55L.jpg")
fairytail100 = Manga(name = "Fairy Tail 100 Years Quest", url= "https://www.scan-vf.net/fairy-tail-100-years-quest", cover="https://pbs.twimg.com/media/FMbhQ8DXoAAFVgI?format=jpg&name=large")
onepunchman = Manga(name = "One Punch Man", url= "https://www.scan-vf.net/one-punch-man", cover="https://static.fnac-static.com/multimedia/Images/FR/NR/16/ec/bf/12577814/1507-1/tsp20210803073801/One-Punch-Man.jpg")
hxh = Manga(name = "Hunter X Hunter", url= "https://www.scan-vf.net/hunter-x-hunter", cover="https://images-na.ssl-images-amazon.com/images/I/81gU6RgiRvL.jpg")
dragon_ball_super = Manga(name = "Dragon Ball Super", url= "https://www.scan-vf.net/dragon-Ball-Super", cover="https://www.db-z.com/wp-content/uploads/2021/11/cover-tome-17-dragon-ball-super-hd.jpg")
the_great_mage = Manga(name = "The Great Mage Returns After 4000 Years", url= "https://manga1st.online/manga/the-great-mage-returns-after-4000-years/", cover="https://static.boxnovel.me/images/the-great-mage-returns-after-4000-years.png")
the_legend = Manga(name = "The Legend of the Northern Blade", url= "https://manga1st.online/manga/legend-of-the-northern-blade/", cover="https://cdn.myanimelist.net/images/manga/3/248937l.jpg")
nano = Manga(name = "Nano Machine", url= "https://manga1st.online/manga/nano-machine/", cover="https://i.pinimg.com/originals/e4/45/54/e44554743cf4d76741a53eb62526e655.png")
blue_lock = Manga(name = "Blue Lock", url= "https://manga1st.online/manga/blue-lock/", cover="https://images-na.ssl-images-amazon.com/images/I/51BkrRCUzeL._SX328_BO1,204,203,200_.jpg")
the_beginning = Manga(name = "The Beginning After The End", url= "https://manga1st.online/manga/the-beginning-after-the-end/", cover="https://pbs.twimg.com/media/FIBd5T0VUAEABDR.jpg")


#J'ajoute juste le dernier élément :) Ne pas ajouter tout le reste en mode liste
db.session.add_all([one_piece,boruto,fairytail100,onepunchman,hxh,dragon_ball_super
                    ,the_great_mage,the_legend,nano,blue_lock,the_beginning]) 

db.session.commit()

c:\Users\cleme\Documents\GitHub\Manwhen\venv\lib\site-packages\flask_sqlalchemy\__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(


In [3]:
def scan_vf_scrap():

    html_text = requests.get(manga.url).text
    soup = BeautifulSoup(html_text, 'html.parser')

    #Je vais chercher les informations sur la page du manga en question
    online_chapters = soup.find_all('h5', {'class' : 'chapter-title-rtl'} )

    #Je cherche toutes les info (name, url, title)
    for num_chap in range(len(online_chapters)):

        last_chap_url = online_chapters[num_chap].find('a')['href']
        last_chap_title = online_chapters[num_chap].find('em').getText()

        last_chap_number_process = online_chapters[num_chap].find('a').getText()
        last_chap_number_process = last_chap_number_process.split()
        last_chap_number = last_chap_number_process[len(last_chap_number_process)-1]

        state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()

        if state is None :
            var = MangaChapter(chapter_number = last_chap_number,
                    chapter_url = last_chap_url, 
                    chapter_name = last_chap_title, 
                    manga = manga)
            db.session.add(var)
            

    db.session.commit()

def manga1st_scrap():

    # J'itère sur tous les manga
    html_text = requests.get(manga.url).text
    soup = BeautifulSoup(html_text, 'html.parser')

    #Je vais chercher les informations sur la page du manga en question
    online_chapters = soup.find_all('li', {'class' : 'wp-manga-chapter'} )
    
    #Je cherche toutes les info (name, url, title)
    for num_chap in range(len(online_chapters)):
        last_chap_url = online_chapters[num_chap].find('a')['href']
        #Il n'y a pas le titre donc je prends "Chapitre x"
        last_chap_title = online_chapters[num_chap].find('a').getText()

        last_chap_number_process = online_chapters[num_chap].find('a').getText()
        last_chap_number_process = last_chap_number_process.split()
        last_chap_number = last_chap_number_process[len(last_chap_number_process)-1]

        state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()
        
        if state is None :
            var = MangaChapter(chapter_number = last_chap_number,
                    chapter_url = last_chap_url, 
                    chapter_name = last_chap_title, 
                    manga = manga)
            db.session.add(var)
            

    db.session.commit()


functions_scrap = {
    "scan-vf": scan_vf_scrap,
    "manga1st": manga1st_scrap
}

In [4]:
from app import Manga, MangaChapter, db
import requests
from bs4 import BeautifulSoup
import tldextract

for manga in Manga.query.all():
    domain_url = tldextract.extract(manga.url)
    functions_scrap[domain_url.domain]()

C:\Users\cleme\AppData\Local\Temp\ipykernel_1340\1645181432.py:54: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()


In [ ]:
from app import Manga, MangaChapter, db
import requests
from bs4 import BeautifulSoup

for manga in Manga.query.all():

  # J'itère sur tous les manga
  html_text = requests.get(manga.url).text
  soup = BeautifulSoup(html_text, 'html.parser')

  #Je vais chercher les informations sur la page du manga en question
  online_chapters = soup.find_all('h5', {'class' : 'chapter-title-rtl'} )

  #Je cherche toutes les info (name, url, title)
  for num_chap in range(len(online_chapters)):

    last_chap_url = online_chapters[num_chap].find('a')['href']
    last_chap_title = online_chapters[num_chap].find('em').getText()

    last_chap_number_process = online_chapters[num_chap].find('a').getText()
    last_chap_number_process = last_chap_number_process.split()
    last_chap_number = last_chap_number_process[len(last_chap_number_process)-1]

    state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()

    if state is None :
        var = MangaChapter(chapter_number = last_chap_number,
                   chapter_url = last_chap_url, 
                   chapter_name = last_chap_title, 
                   manga = manga)
        db.session.add(var)
        

db.session.commit()

In [38]:
from app import db

db.create_all()  

In [ ]:
#cover_add = Manga.query.all()[0]
#cover_add.cover = 'https://www.scan-vf.net/uploads/manga/dragon-Ball-Super/cover/cover_250x350.jpg'
#db.session.commit()

In [5]:
from app import Manga, MangaChapter, db
db.session.query(Manga.id).filter_by(
        name='Two Piece').scalar() is not None

False

In [ ]:
from app import Manga, MangaChapter, db

manga_test = Manga.query.get(1)
#manga_test.id

manga_test_2 = db.session.query(Manga.id).filter_by(name='Hunter  Hunter').scalar() is not None
print(manga_test_2)

False


In [2]:
from app import Manga, MangaChapter, db
import requests
from bs4 import BeautifulSoup

for manga in Manga.query.all():

  # J'itère sur tous les manga
  html_text = requests.get(manga.url).text
  soup = BeautifulSoup(html_text, 'html.parser')

  #Je vais chercher les informations sur la page du manga en question
  online_chapters = soup.find_all('h5', {'class' : 'chapter-title-rtl'} )

  #Je cherche toutes les info (name, url, title)
  for num_chap in range(len(online_chapters)):

    last_chap_url = online_chapters[num_chap].find('a')['href']
    last_chap_title = online_chapters[num_chap].find('em').getText()

    last_chap_number_process = online_chapters[num_chap].find('a').getText()
    last_chap_number_process = last_chap_number_process.split()
    last_chap_number = last_chap_number_process[len(last_chap_number_process)-1]

    state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()

    if state is None :
        var = MangaChapter(chapter_number = last_chap_number,
                   chapter_url = last_chap_url, 
                   chapter_name = last_chap_title, 
                   manga = manga)
        db.session.add(var)
        

db.session.commit()
    


C:\Users\cleme\AppData\Local\Temp\ipykernel_4552\3145937350.py:24: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  state = MangaChapter.query.filter_by(chapter_number=last_chap_number, manga_id = manga.id).first()


In [9]:
import tldextract
url_test = 'https://manga1st.online/manga/the-beginning-after-the-end/'
domain_url = tldextract.extract(url_test)
domain_url.domain

def aa():
    print('aaaa')

def bb():
    print('bbbb')

functions_scrap = {
    "scan-vf": aa,
    "manga1st": bb
}


if domain_url.domain in functions_scrap:
    functions_scrap[domain_url.domain]()


bbbb


In [36]:
#Pour supprimer une ligne Manga (ici la 7e)
Manga.query.filter(Manga.id == 10).delete()
db.session.commit()

In [34]:
import requests
from bs4 import BeautifulSoup

manga_url = 'https://manga1st.online/manga/the-great-mage-returns-after-4000-years/'
html_text = requests.get(manga_url).text
soup = BeautifulSoup(html_text, 'html.parser')
online_chapters = soup.find_all('li', {'class' : 'wp-manga-chapter'} )
len(online_chapters)

123

In [20]:
Manga.query.order_by(Manga.id.desc()).first()

manga('The Great Mage Returns After 4000 Years','https://manga1st.online/manga/the-great-mage-returns-after-4000-years/','https://static.boxnovel.me/images/the-great-mage-returns-after-4000-years.png')

In [16]:
db.session.rollback()

In [ ]:
str = 'One Piece | 1045'

In [ ]:
var = MangaChapter.query.filter(MangaChapter.chapter_number<=400,MangaChapter.manga_id=='1').all()
type(var)

list

In [87]:
manga_num_clicked = '101'
manga_id_current = '1'

state_manga_num_clicked = (MangaChapter.query.filter(MangaChapter.chapter_number==manga_num_clicked,MangaChapter.manga_id=='1').first()).chapter_viewed

#Si le chapitre est déjà lu, je le marque en non lu
if state_manga_num_clicked == True:
    state_manga_num_clicked = False
        
    db.session.commit()

#Si le chapitre est pas encore lu
else:
    MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)


    #Je regarde déjà si parmi les précédents, il existe des chapitres non lu
    number_chap_not_viewed = MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).all()

    if len(number_chap_not_viewed) == 0:
        #Changer l'image pour dire que le chap est désormais lu
        #Modifier la valeur en base
        MangaChapter.query.filter(MangaChapter.chapter_number==manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)
    
    else:
        #Demander si je mets à jour les précédents        
        ## if oui à tout cocher
        MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)
        ## si non, je modifie juste celui coché 
        MangaChapter.query.filter(MangaChapter.chapter_number==manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)



In [84]:
var2 = MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id=='1',MangaChapter.chapter_viewed==False).all()
var2

[]

In [85]:
MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id=='1',MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)

0

In [9]:
db.session.rollback()

In [ ]:
manga_num_clicked = '101'
manga_id_current = '1'

state_manga_num_clicked = (MangaChapter.query.filter(MangaChapter.chapter_number==manga_num_clicked,MangaChapter.manga_id=='1').first()).chapter_viewed

#Si le chapitre est déjà lu, je le marque en non lu
if state_manga_num_clicked == True:
    state_manga_num_clicked = False
        
    db.session.commit()

#Si le chapitre est pas encore lu
else:
    MangaChapter.query.filter(MangaChapter.chapter_number<=manga_num_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)


In [97]:
mgc = MangaChapter.query.all()
mgc[2].chapter_viewed

False

In [8]:

abc = MangaChapter.query.filter(MangaChapter.chapter_number<='1000',MangaChapter.manga_id=='1',MangaChapter.chapter_viewed==True)
len(abc)

TypeError: object of type 'BaseQuery' has no len()

In [107]:
mangachapter = MangaChapter.query.all()
manga = Manga.query.get(1)

manga.manga_chapters


for chapter in manga.manga_chapters :
    if mangachapter[1040].chapter_viewed == True : 
        

IndentationError: expected an indented block after 'if' statement on line 8 (28157393.py, line 9)

False

In [10]:
mangachapter = MangaChapter.query.get('100')
mangachapter

manga_chapter('Entrée en scène de Kawamatsu le Kappa','948.0000000000','https://www.scan-vf.net/one_piece/chapitre-948','True')

In [115]:
manga = (Manga.query.get(1)).id
print(type(manga))

<class 'int'>


In [119]:
manga_chapter_view_clicked = int('1041')
type(manga_chapter_view_clicked)
print(manga_chapter_view_clicked)


1041


In [121]:
manga_chapter_view_clicked = '1040'
manga_id_current = '1'

state_manga_chapter_view_clicked = (MangaChapter.query.filter(MangaChapter.chapter_number==manga_chapter_view_clicked,MangaChapter.manga_id=='1').first()).chapter_viewed
print(state_manga_chapter_view_clicked)

False


In [123]:
MangaChapter.query.filter(MangaChapter.chapter_number<=manga_chapter_view_clicked,MangaChapter.manga_id==manga_id_current,MangaChapter.chapter_viewed==False).update({MangaChapter.chapter_viewed: True},synchronize_session= False)
db.session.commit()


In [30]:
#Je regarde la longueur des chap non lu dans la base
indice_manga = '1' # '1' = One Piece

len(MangaChapter.query.filter( MangaChapter.chapter_viewed == False , MangaChapter.manga_id== indice_manga).all())

1

In [126]:
mangachapter = MangaChapter.query.all()
mangachapter[1000.5]

TypeError: list indices must be integers or slices, not float

In [33]:
mangas_list = Manga.query.all()
len(mangas_list)

6

In [ ]:
True_count_list = ['1','2','3','4','5']
False_count_list = ['101','102','103','104','105']

